<a href="https://colab.research.google.com/github/neowalter/CS658/blob/main/HW7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# count words

In [ ]:

import os       
!apt-get remove -y java*
def install_java():
    !apt-get install -y openjdk-8-jdk-headless -qq > /dev/null      
    os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"     
    !java -version       

install_java()

In [9]:
!export JAVA_HOME=/usr/lib/jvm/java-8-openjdk-amd64/jre
!pip install pyspark
import pyspark
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql import SQLContext

spark = SparkSession.builder \
   .master("local") \
   .appName("Wordcount & Friends Test") \
   .config("spark.executor.memory", "2gb") \
   .getOrCreate()
   
sc = spark.sparkContext
sqlContext = SQLContext(sc)

sc.version

'3.0.1'

In [10]:
!wget https://raw.githubusercontent.com/neowalter/CS658/main/wordcount_test_text
!cat wordcount_test_text


--2020-11-03 05:22:57--  https://raw.githubusercontent.com/neowalter/CS658/main/wordcount_test_text
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14385 (14K) [text/plain]
Saving to: ‘wordcount_test_text’

wordcount_test_text 100%[===================>]  14.05K  --.-KB/s    in 0.01s   

2020-11-03 05:22:57 (1.19 MB/s) - ‘wordcount_test_text’ saved [14385/14385]

How to build a better business model by blockchain
Abstract

At present, we are living in a world that the surroundings are changing dramatically every day. Since the blockchain has been invented, it already self-upgraded three versions (Spithoven, 2019). Blockchain has been treated as a technological invention that can change the world, the main features include cryptography and cybersecurity. Many compa

In [13]:
rdd0 = sc.textFile("wordcount_test_text")
rdd0.take(20)

['How to build a better business model by blockchain',
 'Abstract',
 '',
 'At present, we are living in a world that the surroundings are changing dramatically every day. Since the blockchain has been invented, it already self-upgraded three versions (Spithoven, 2019). Blockchain has been treated as a technological invention that can change the world, the main features include cryptography and cybersecurity. Many companies already used it to deploy their new business model in many different fields (Zhang, Xue & Liu, 2019). The security and privacy of blockchains which are shown by the current applications proved that this advanced technology can help with building a better model.',
 '',
 'Introduction',
 '',
 'Not far from now, on May 22, 2010, a programmer bought two large pizzas for 10,000 bitcoins. They were only worth about 30 dollars at the time (Wong, 2018). Now worth 80 million dollars, none of the companies have this rising price in the stock market like this.',
 '',
 'Since th

In [14]:
word_counts = rdd0.flatMap(lambda line: line.split()).map(lambda word: (word, 1)).reduceByKey(lambda a, b: (a + b)).map(lambda x:(x[1],x[0]))

word_counts.take(30)

[(3, 'How'),
 (48, 'to'),
 (4, 'build'),
 (40, 'a'),
 (6, 'better'),
 (9, 'business'),
 (6, 'model'),
 (11, 'by'),
 (40, 'blockchain'),
 (1, 'Abstract'),
 (1, 'At'),
 (1, 'present,'),
 (1, 'we'),
 (13, 'are'),
 (1, 'living'),
 (21, 'in'),
 (1, 'world'),
 (14, 'that'),
 (129, 'the'),
 (1, 'surroundings'),
 (2, 'changing'),
 (2, 'dramatically'),
 (4, 'every'),
 (2, 'day.'),
 (2, 'Since'),
 (8, 'has'),
 (7, 'been'),
 (1, 'invented,'),
 (18, 'it'),
 (4, 'already')]

# Get name from wiki table

In [22]:
import requests  
from bs4 import BeautifulSoup 
import pandas as pd
import csv
import time
import matplotlib.pyplot as plt

url1 = "https://en.wikipedia.org/wiki/List_of_most_popular_given_names#Male_names_2"

s = requests.Session()
response = s.get(url1, timeout=10)
response

<Response [200]>

In [23]:
soup = BeautifulSoup(response.content, 'html.parser')
pretty_soup = soup.prettify()

soup.title.string

'List of most popular given names - Wikipedia'

In [24]:
all_tables=soup.find_all('table')
#wikitable sortable mw-collapsible jquery-tablesorter mw-made-collapsible
right_table=soup.find('table', {"class":'wikitable'})
print(right_table)

<table class="wikitable">
<tbody><tr style="background:#a0d0ff;">
<th style="width:300px;">Region (year)</th>
<th>No. 1</th>
<th>No. 2</th>
<th>No. 3</th>
<th>No. 4</th>
<th>No. 5</th>
<th>No. 6</th>
<th>No. 7</th>
<th>No. 8</th>
<th>No. 9</th>
<th>No. 10
</th></tr>
<tr>
<td><a href="/wiki/Algeria" title="Algeria">Algeria</a> (census, 2010)<sup class="reference" id="cite_ref-ine.es_1-0"><a href="#cite_note-ine.es-1">[1]</a></sup></td>
<td><a href="/wiki/Muhammad_(name)" title="Muhammad (name)">Mohamed</a></td>
<td>Abdelkader</td>
<td><a href="/wiki/Ahmad" title="Ahmad">Ahmed</a></td>
<td><a href="/wiki/Muhammad_(name)" title="Muhammad (name)">Mohammed</a></td>
<td><a href="/wiki/Ali_(name)" title="Ali (name)">Ali</a></td>
<td>Rachid</td>
<td>Said</td>
<td><a href="/wiki/Ibrahim_(name)" title="Ibrahim (name)">Brahim</a></td>
<td><a href="/wiki/Omar_(name)" title="Omar (name)">Omar</a></td>
<td>Djamel
</td></tr>
<tr>
<td><a href="/wiki/Egypt" title="Egypt">Egypt</a> (2004, unofficial)<su

In [25]:
for row in right_table.find_all("tr"):
    cells = row.findAll('td')

len(cells)

11

In [26]:
rows = right_table.findAll("tr")
len(rows)

11

In [27]:
header = [th.text.rstrip() for th in rows[0].find_all('th')]
print(header)
print('------------')
print(len(header))

['Region (year)', 'No. 1', 'No. 2', 'No. 3', 'No. 4', 'No. 5', 'No. 6', 'No. 7', 'No. 8', 'No. 9', 'No. 10']
------------
11


In [85]:
remove_items = ['NA', 'Algeria (census, 2010)[1]', 'Egypt (2004, unofficial)[2]', 
                'Egypt (2004, Coptic Christians, unofficial)[2]', 
                'Ethiopia (2004, Orthodox Christians, unofficial)[citation needed]', 
                'Equatorial Guinea (2011)[3]', 
                'Libya[citation needed]', 'Mali (census, 2010)[1]', 'Morocco[4]',
                'South Africa[5] (2015)', 'Tunisia[6]']


lst_data = []
for row in rows[1:]:
    for d in row.select('td'):
        data = d.text.rstrip()
        if data != 'NA':
            lst_data.append(data)

for items in remove_items:
    try:
        lst_data.remove(items)
    except ValueError:
        pass  


['Mohamed', 'Abdelkader', 'Ahmed', 'Mohammed', 'Ali', 'Rachid', 'Said', 'Brahim', 'Omar', 'Djamel', 'Mohamed, Youssef, Ahmed, Mahmoud, Mustafa, Yassin, Taha, Khaled, Hamza, Bilal, Ibrahim, Hassan, Hussein, Karim, Tareq, Abdel-Rahman, Ali, Omar, Halim, Murad, Selim, Abdallah', 'Peter, Pierre, George, John, Mina, Beshoi, Kirollos, Mark, Fadi, Habib', 'Paulos, Petros, Gabreal, Giorgis, Yonas, Yonathan, Abraham, Ammanuel, Markos, Michael, Nahom', 'Manuel', 'Juan', 'Antonio', 'José', 'Mohammed', 'Ahmed', 'Ali', 'Hamza', 'Ibrahim', 'Mahmoud', 'Abdallah', 'Tareq', 'Hassan', 'Khaled', 'Mamadou', 'Moussa', 'Mahamadou', 'Adama', 'Bakary', 'Abdoulaye', 'Modibo', 'Oumar', 'Sekou', 'Souleymane', 'Mohamed', 'Youssef', 'Hamza', 'Omar', 'Amine', 'Karim', 'Mehdi', 'Yassine', 'Zakaria', 'Ali', 'Junior', 'Blessing', 'Gift', 'Bandile', 'Prince', 'Siyabonga', 'Melokuhle', 'Lethabo', 'Banele', 'Samkelo', 'Mehdi', 'Youssef', 'Aziz', 'Karim']
61


# friend detection

In [94]:
!wget "https://raw.githubusercontent.com/neowalter/CS658/main/friends"
rdd = sc.textFile("friends")
!cat friends

--2020-11-03 08:08:37--  https://raw.githubusercontent.com/neowalter/CS658/main/friends
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 466 [text/plain]
Saving to: ‘friends’

friends             100%[===================>]     466  --.-KB/s    in 0s      

2020-11-03 08:08:37 (28.3 MB/s) - ‘friends’ saved [466/466]

Mohamed Omar
Abdelkader Ali
Ahmed Adama
Brahim Adama
Omar George
Youssef George
Ali George
Omar Adama
Halim Ali
Murad George
Selim George
Abdallah Ali
Peter Mina
Pierre Mina
George Adama
John Giorgis
Mina Giorgis
Beshoi George
Kirollos Mina
Mark Youssef
Fadi Mina
Habib Paulos
Petros Gabreal 
Giorgis Giorgis
Yonas Youssef
Yonathan Giorgis
Abraham Ammanuel 
Markos Yonas
Michael Nahom 
Juan Youssef
Antonio Ali
Adama Bakary 
Mohamed Youssef
Ali Markos
Yous

In [95]:
rdd.take(20)

['Mohamed Omar',
 'Abdelkader Ali',
 'Ahmed Adama',
 'Brahim Adama',
 'Omar George',
 'Youssef George',
 'Ali George',
 'Omar Adama',
 'Halim Ali',
 'Murad George',
 'Selim George',
 'Abdallah Ali',
 'Peter Mina',
 'Pierre Mina',
 'George Adama',
 'John Giorgis',
 'Mina Giorgis',
 'Beshoi George',
 'Kirollos Mina',
 'Mark Youssef']

In [97]:
rdd1=rdd.map(lambda x: x.split()).union(rdd.map(lambda x: x.split()[::-1]))

lst = rdd1.filter(lambda x: x[0] == 'Ali').map(lambda x: x[1]).collect()

rdd2 = rdd1.filter(lambda x: x[0] in lst).map(lambda x: x[1]). \
    filter(lambda x: x != 'Ali' and x not in lst). \
    map(lambda x: (x, 1)).reduceByKey(lambda a, b: a + b). \
    map(lambda x: (x[1], x[0])).sortByKey(ascending = False)

for x, y in rdd2.collect():
   print ("The stranger {} has {} common friends with Ali".format(y, x))


The stranger Adama has 1 common friends with Ali
The stranger Yonas has 1 common friends with Ali
The stranger Omar has 1 common friends with Ali
The stranger Youssef has 1 common friends with Ali
The stranger Murad has 1 common friends with Ali
The stranger Beshoi has 1 common friends with Ali
The stranger Selim has 1 common friends with Ali
